# Set up

In [ ]:
!pip install parse
!pip install levenshtein

# process results

In [ ]:
import pandas as pd

result_data = pd.read_csv("/content/model.csv")
result_data

In [ ]:
result_list = result_data["Classes"].to_list()

import parse
import pprint
pp = pprint.PrettyPrinter(indent=4)

result_classes = []
for index, result in enumerate(result_list):
  # Enumeration class: the substring after "Enumeration:/n" and before "\nClasses:"
  enumeration = result.partition("Enumeration:\n")[2].partition("\nClasses:")[0]
  # Classes: the substring after "Classes:\n"
  classes = result.partition("Classes:\n")[2]
  # split string to list of sub-string, remove empty string
  result = (enumeration + "\n" + classes).splitlines()
  result = [i for i in result if (i and i != "None" and i != " " and i != "")]
  result_classes.append(result)

# we do not need the first two classes for evaluation: BTMS, H2S and the last class, H2S shot
result_classes = result_classes[2:9]
pp.pprint(result_classes)

[   [   'Interval(weekly, monthly, everyHalfYear, yearly)',
        'AccessType(reservable, walkIn, dropOff)',
        'DayOfWeek(Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, '
        'Sunday)',
        'LabTracker()',
        'Person(string lastName, string firstName, string address, string '
        'phoneNumber)',
        'abstract PersonRole(idNumber)',
        'Patient(string dateOfBirth)',
        'Doctor(string signature)',
        'Requisition(string effectiveDate, int repetitionCount, Interval '
        'repetitionInterval)',
        'TestResult(boolean negative, string report)',
        'SpecificTest(Data date)',
        'Appointment(string confirmation, Date date, string startTime, string '
        'endTime)',
        'BusinessHour(DayOfWeek: dayOfWeek, string startTime, string endTime)',
        'Lab(string registrationNumber, string name, string address, boolean '
        'changeCancelFee)',
        'Test(string name, string duration)',
        'TestType(string

In [ ]:
print(len(result_classes))

7


# 0-shot

In [ ]:
import pandas as pd

zero_shot_data = pd.read_csv("/content/gpt3_zero_shot.csv")
zero_shot_data

In [ ]:
zero_shot_generated_list = zero_shot_data["generated_text"].to_list()
zero_shot_generated_list = zero_shot_generated_list[2:9]

print(len(zero_shot_generated_list))

7


In [ ]:
import parse

generated_classes = []
for index, result in enumerate(zero_shot_generated_list):
  # Enumeration class: the substring after "Enumeration:/n" and before "\nClasses:"
  if "Enumerations:" in result:
    enumeration = result.partition("Enumerations:")[2].partition("Class")[0]
  else:
    enumeration = result.partition("Enumeration:")[2].partition("Class")[0]
  
  # Classes: the substring after "Classes:\n"
  if "Class:\n" in result:
    classes = result.partition("Class:\n")[2].partition("Relationships:")[0]
  else:
    classes = result.partition("Classes:")[2].partition("Relationships:")[0]
  
  # split string to list of string, remove empty string
  classes = (enumeration + "\n" + classes).splitlines()
  result = [i for i in classes if (i and i != "None" and i != " " and i != "")]
  generated_classes.append(result)

generated_classes

[['TestGroup(bloodTest, ultrasoundExam)',
  'Doctor(int practitionerNumber, String fullName, String address, String phoneNumber, Image signature)',
  'Patient(String healthNumber, String firstName, String lastName, Date dateOfBirth, String address, String phoneNumber)',
  'Requisition(Date validFrom, Doctor doctor, Patient patient)',
  'Test(int duration, TestGroup testGroup)',
  'Lab(String address, int businessHours, int fee, int registrationNumber)',
  'Appointment(String confirmationNumber, Date date, int startTime, int endTime, Lab lab)'],
 ['Events(birthday party, graduation party...)',
  'Organizer(string firstName, string lastName, string emailAddress, string postalAddress, string phoneNumber, string password)',
  'Attendee(string firstName, string lastName, string emailAddress, string password)',
  'Event(string eventType, DateTime startDateTime, DateTime endDateTime, string occasion, Location location)',
  'Location(string name, string address)',
  'Checklist(string task, boo

In [ ]:
def convert_to_dict(result):
  classes = {}
  for i in range(len(result)):
    class_name = result[i].split("(")[0]
    class_attribute = result[i].split("(")[1].replace(")", "")
    classes[class_name]= class_attribute
  return classes

def find_number_exact_match(l1, l2):
  number = 0
  for c1 in l1:
    for c2 in l2:
      if ratio(c1, c2) >= 0.9:
        # found
        print("Attribute: ",  c1, c2)
        number += 1
        l2.remove(c2)
  return number

In [ ]:
from Levenshtein import distance
from Levenshtein import ratio

number_class_exact_match = 0
number_class_false_positive = 0   # generated but is not part of the solution
number_class_false_negative = 0   # is part of the solution but is not being generated

number_attribute_exact_match = 0
number_attribute_false_positive = 0   # generated but is not part of the solution
number_attribute_false_negative = 0   # is part of the solution but is not being generated

for index in range(len(result_classes)):
  print(index)
  solution = convert_to_dict(result_classes[index])
  prediction = convert_to_dict(generated_classes[index])
  
  solution_class = list(solution.keys())
  prediction_class = list(prediction.keys())
  
  class_current_match = 0
  for c1 in solution_class:
    for c2 in prediction_class:
      # consider as exact match if the ratio > 0.9  
      if ratio(c1, c2) >= 0.9:
        # found
        print("Class: ", c1, c2)
        class_current_match += 1
        prediction_class.remove(c2)

        # find number of exact match for attribute
        attribute_current_match = 0
        attribute_solution = list(solution[c1].split(","))
        attribute_generated = list(prediction[c2].split(","))

        for a1 in attribute_solution:
          for a2 in attribute_generated:
            # consider as exact match if the ratio > 0.9  
            if a1 != "" and ratio(a1, a2) >= 0.9:
              # found
              print("Attribute: ", a1, a2)
              attribute_current_match += 1
              attribute_generated.remove(a2)
        
        number_attribute_exact_match += attribute_current_match
        number_attribute_false_positive += len(attribute_generated)
        number_attribute_false_negative += (len(attribute_solution) - attribute_current_match)

  number_class_false_positive += len(prediction_class)
  number_class_false_negative += (len(solution_class) - class_current_match)
  number_class_exact_match += class_current_match

0
Class:  Patient Patient
Class:  Doctor Doctor
Class:  Requisition Requisition
Class:  Appointment Appointment
Attribute:   Date date  Date date
Class:  Lab Lab
Class:  Test Test
1
Class:  Organizer Organizer
Attribute:   string phoneNumber  string phoneNumber
Class:  Attendee Attendee
Class:  Location Location
Attribute:  string name string name
Attribute:   string address  string address
Class:  Event Events
2
Class:  Position Position
Attribute:  GK GK
Attribute:   LB  LB
Class:  Recommendation Recommendation
Class:  HeadCoach HeadCoach
Class:  Director Director
Class:  Scout Scout
3
4
Class:  Tutor Tutor
Attribute:  string bankAccount  string bankAccount
Class:  Student Student
Class:  TutoringSession TutoringSession
5
Class:  User User
Attribute:  string username String username
Attribute:   string password  String password
Class:  Level Level
Class:  Game Game
Attribute:  string name String name
Class:  Block Block
Attribute:   int points  int points
6
Class:  Game Game
Class:  

In [ ]:
print("number_class_exact_match   ", number_class_exact_match)
print("number_class_false_positive    ", number_class_false_positive)
print("number_class_false_negative  ", number_class_false_negative)

print("number_attribute_exact_match   ", number_attribute_exact_match)
print("number_attribute_false_positive    ", number_attribute_false_positive)
print("number_attribute_false_negative  ", number_attribute_false_negative)

# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN) 
# F1 = (precision * recall) / (precision + recall)

class_precision = number_class_exact_match / (number_class_exact_match + number_class_false_positive)
class_recall = number_class_exact_match / (number_class_exact_match + number_class_false_negative)
class_f1 = (class_precision * class_recall) / (class_precision + class_recall)

attribute_precision = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_positive)
attribute_recall = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_negative)
attribute_f1 = (attribute_precision * attribute_recall) / (attribute_precision + attribute_recall)

print("class_precision: ", class_precision)
print("class_recall: ", class_recall)
print("class_f1: ", class_f1)

print("attribute_precision: ", attribute_precision)
print("attribute_recall: ", attribute_recall)
print("attribute_f1: ", attribute_f1)

number_class_exact_match    27
number_class_false_positive     21
number_class_false_negative   84
number_attribute_exact_match    11
number_attribute_false_positive     85
number_attribute_false_negative   48
class_precision:  0.5625
class_recall:  0.24324324324324326
class_f1:  0.169811320754717
attribute_precision:  0.11458333333333333
attribute_recall:  0.1864406779661017
attribute_f1:  0.07096774193548387


# 1-shot use BTMS as example

In [ ]:
one_shot_btms_data = pd.read_csv("/content/gpt3_one_shot_btms.csv")
one_shot_btms_data

In [ ]:
one_shot_btms_generated_list = one_shot_btms_data["generated_text"].to_list()
one_shot_btms_generated_list = one_shot_btms_generated_list[1:8]

print(len(one_shot_btms_generated_list))

7


In [ ]:
import parse

generated_classes = []
for index, result in enumerate(one_shot_btms_generated_list):
  # Enumeration class: the substring after "Enumeration:/n" and before "\nClasses:"
  if "Enumerations:" in result:
    enumeration = result.partition("Enumerations:")[2].partition("Class")[0]
  else:
    enumeration = result.partition("Enumeration:")[2].partition("Class")[0]
  
  # Classes: the substring after "Classes:\n"
  if "Class:\n" in result:
    classes = result.partition("Class:\n")[2].partition("Relationships:")[0]
  else:
    classes = result.partition("Classes:")[2].partition("Relationships:")[0]
  
  # split string to list of string, remove empty string
  classes = (enumeration + "\n" + classes).splitlines()
  result = [i for i in classes if (i and i != "None" and i != " " and i != "")]
  generated_classes.append(result)

generated_classes

[['LabTracker()',
  'Doctor(string practitionerNumber, string signature, string fullName, string address, string phoneNumber)',
  'Patient(string healthNumber, string firstName, string lastName, Date dateOfBirth, string address, string phoneNumber)',
  'TestGroup(string name)',
  'Test(string name, int duration)',
  'Requisition(Date validDate)',
  'Appointment(Date date, Time startTime, Time endTime, string labName, string registrationNumber, int fee)'],
 ['CelO()',
  'Organizer(string FirstName, string LastName, string EmailAddress, string PostalAddress, string PhoneNumber, string Password)',
  'Event(string KindOfEvent, DateTime StartTime, DateTime EndTime, string Occasion, string Location)',
  'Attendee(string FirstName, string LastName, string EmailAddress, string Password)',
  'InvitationStatus(string Status)',
  'EventChecklist(string Task, boolean Done, boolean Applicable)',
  'AttendeeChecklist(string Task)'],
 ['TeamSportsScoutingSystem()',
  'PlayerProfile(string targetPosit

In [ ]:
from Levenshtein import distance
from Levenshtein import ratio

number_class_exact_match = 0
number_class_false_positive = 0   # generated but is not part of the solution
number_class_false_negative = 0   # is part of the solution but is not being generated

number_attribute_exact_match = 0
number_attribute_false_positive = 0   # generated but is not part of the solution
number_attribute_false_negative = 0   # is part of the solution but is not being generated

for index in range(len(result_classes)):
  print(index)
  solution = convert_to_dict(result_classes[index])
  prediction = convert_to_dict(generated_classes[index])
  
  solution_class = list(solution.keys())
  prediction_class = list(prediction.keys())
  
  class_current_match = 0
  for c1 in solution_class:
    for c2 in prediction_class:
      # consider as exact match if the ratio > 0.9  
      if ratio(c1, c2) >= 0.9:
        # found
        print("Class: ", c1, c2)
        class_current_match += 1
        prediction_class.remove(c2)

        # find number of exact match for attribute
        attribute_current_match = 0
        attribute_solution = list(solution[c1].split(","))
        attribute_generated = list(prediction[c2].split(","))

        for a1 in attribute_solution:
          for a2 in attribute_generated:
            # consider as exact match if the ratio > 0.9  
            if a1 != "" and ratio(a1, a2) >= 0.9:
              # found
              print("Attribute: ", a1, a2)
              attribute_current_match += 1
              attribute_generated.remove(a2)
        
        number_attribute_exact_match += attribute_current_match
        number_attribute_false_positive += len(attribute_generated)
        number_attribute_false_negative += (len(attribute_solution) - attribute_current_match)

  number_class_false_positive += len(prediction_class)
  number_class_false_negative += (len(solution_class) - class_current_match)
  number_class_exact_match += class_current_match

0
Class:  LabTracker LabTracker
Class:  Patient Patient
Class:  Doctor Doctor
Attribute:  string signature  string signature
Class:  Requisition Requisition
Class:  Appointment Appointment
Attribute:   Date date Date date
Class:  Test Test
Attribute:  string name string name
1
Class:  CelO CelO
Class:  Organizer Organizer
Attribute:   string phoneNumber  string PhoneNumber
Class:  Attendee Attendee
Class:  Event Event
Attribute:  string occasion  string Occasion
2
Class:  Player Player
Class:  ScoutingAssignmnet ScoutingAssignment
Class:  PlayerProfile PlayerProfile
3
4
Class:  OTS OTS
Class:  Subject Subject
Attribute:  string name string name
Class:  Tutor Tutor
Attribute:  string bankAccount  string bankAccount
Class:  TutoringOffer TutoringOffer
Class:  Student Student
Class:  TutoringRequest TutoringRequest
Class:  TutoringSession TutoringSession
Class:  Payment Payment
5
Class:  User User
Attribute:  string username string username
Attribute:   string password  string password
Cl

In [ ]:
print("number_class_exact_match   ", number_class_exact_match)
print("number_class_false_positive    ", number_class_false_positive)
print("number_class_false_negative  ", number_class_false_negative)

print("number_attribute_exact_match   ", number_attribute_exact_match)
print("number_attribute_false_positive    ", number_attribute_false_positive)
print("number_attribute_false_negative  ", number_attribute_false_negative)

# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN) 
# F1 = (precision * recall) / (precision + recall)

class_precision = number_class_exact_match / (number_class_exact_match + number_class_false_positive)
class_recall = number_class_exact_match / (number_class_exact_match + number_class_false_negative)
class_f1 = (class_precision * class_recall) / (class_precision + class_recall)

attribute_precision = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_positive)
attribute_recall = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_negative)
attribute_f1 = (attribute_precision * attribute_recall) / (attribute_precision + attribute_recall)

print("class_precision: ", class_precision)
print("class_recall: ", class_recall)
print("class_f1: ", class_f1)

print("attribute_precision: ", attribute_precision)
print("attribute_recall: ", attribute_recall)
print("attribute_f1: ", attribute_f1)

number_class_exact_match    31
number_class_false_positive     19
number_class_false_negative   80
number_attribute_exact_match    12
number_attribute_false_positive     64
number_attribute_false_negative   42
class_precision:  0.62
class_recall:  0.27927927927927926
class_f1:  0.19254658385093168
attribute_precision:  0.15789473684210525
attribute_recall:  0.2222222222222222
attribute_f1:  0.09230769230769231


# 1-shot use H2S short as example

In [ ]:
one_shot_h2s_data = pd.read_csv("/content/gpt3_one_shot_h2sshort.csv")
one_shot_h2s_data

In [ ]:
one_shot_h2ssort_generated_list = one_shot_h2s_data["generated_text"].to_list()
one_shot_h2ssort_generated_list = one_shot_h2ssort_generated_list[2:9]

print(len(one_shot_h2ssort_generated_list))

7


In [ ]:
import parse

generated_classes = []
for index, result in enumerate(one_shot_h2ssort_generated_list):
  # Enumeration class: the substring after "Enumeration:/n" and before "\nClasses:"
  if "Enumerations:" in result:
    enumeration = result.partition("Enumerations:")[2].partition("Class")[0]
  else:
    enumeration = result.partition("Enumeration:")[2].partition("Class")[0]
  
  # Classes: the substring after "Classes:\n"
  if "Class:\n" in result:
    classes = result.partition("Class:\n")[2].partition("Relationships:")[0]
  else:
    classes = result.partition("Classes:")[2].partition("Relationships:")[0]
  
  # split string to list of string, remove empty string
  classes = (enumeration + "\n" + classes).splitlines()
  result = [i for i in classes if (i and i != "None" and i != " " and i != "")]
  generated_classes.append(result)

generated_classes

[['LabTracker()',
  'Doctor(string practitionerNo, string signature, string fullName, string address, string phoneNumber)',
  'Patient(int healthNumber, string firstName, string lastName, Date dateOfBirth, string address, string phoneNumber)',
  'TestRequisition(Date validFrom, string name, string address, string phoneNumber)',
  'Test(string group, int duration, boolean appointmentRequired, boolean walkInOnly, boolean sampleDropOff)',
  'Lab(string address, string businessHours, double changeCancellationFee)',
  'Appointment(int confirmationNo, string labName, string labRegistrationNo)'],
 ['CelO()',
  'Event(string type, Date startTime, Date endTime, string occasion, string location)',
  'Organizer(string firstName, string lastName, string emailAddress, string postalAddress, string phoneNumber, string password)',
  'Attendee (string firstName, string lastName, string emailAddress, string password)',
  'Task(string name, boolean toBeDone, boolean done, boolean notApplicable)'],
 ['Tea

In [ ]:
from Levenshtein import distance
from Levenshtein import ratio

number_class_exact_match = 0
number_class_false_positive = 0   # generated but is not part of the solution
number_class_false_negative = 0   # is part of the solution but is not being generated

number_attribute_exact_match = 0
number_attribute_false_positive = 0   # generated but is not part of the solution
number_attribute_false_negative = 0   # is part of the solution but is not being generated

for index in range(len(result_classes)):
  print(index)
  solution = convert_to_dict(result_classes[index])
  prediction = convert_to_dict(generated_classes[index])
  
  solution_class = list(solution.keys())
  prediction_class = list(prediction.keys())
  
  class_current_match = 0
  for c1 in solution_class:
    for c2 in prediction_class:
      # consider as exact match if the ratio > 0.9  
      if ratio(c1, c2) >= 0.9:
        # found
        print("Class: ", c1, c2)
        class_current_match += 1
        prediction_class.remove(c2)

        # find number of exact match for attribute
        attribute_current_match = 0
        attribute_solution = list(solution[c1].split(","))
        attribute_generated = list(prediction[c2].split(","))

        for a1 in attribute_solution:
          for a2 in attribute_generated:
            # consider as exact match if the ratio > 0.9  
            if a1 != "" and ratio(a1, a2) >= 0.9:
              # found
              print("Attribute: ", a1, a2)
              attribute_current_match += 1
              attribute_generated.remove(a2)
        
        number_attribute_exact_match += attribute_current_match
        number_attribute_false_positive += len(attribute_generated)
        number_attribute_false_negative += (len(attribute_solution) - attribute_current_match)

  number_class_false_positive += len(prediction_class)
  number_class_false_negative += (len(solution_class) - class_current_match)
  number_class_exact_match += class_current_match

0
Class:  LabTracker LabTracker
Class:  Patient Patient
Class:  Doctor Doctor
Attribute:  string signature  string signature
Class:  Appointment Appointment
Class:  Lab Lab
Attribute:   string address string address
Class:  Test Test
1
Class:  CelO CelO
Class:  Organizer Organizer
Attribute:   string phoneNumber  string phoneNumber
Class:  Attendee Attendee 
Class:  Event Event
Attribute:  string occasion  string occasion
Attribute:   date startTime  Date startTime
Attribute:   date endTime  Date endTime
2
Class:  Player Player
Class:  HeadCoach HeadCoach
Class:  Director Director
Class:  ScoutReport ScoutReport
Attribute:  string pro string pros
Attribute:   string con  string cons
3
4
Class:  OTS OTS
Class:  Subject Subject
Class:  Tutor Tutor
Attribute:  string bankAccount  string bankAccount
Class:  Student Student 
Class:  TutoringSession TutoringSession
5
Class:  User User
Attribute:  string username string username
Attribute:   string password  string password
Class:  Level Leve

In [ ]:
print("number_class_exact_match   ", number_class_exact_match)
print("number_class_false_positive    ", number_class_false_positive)
print("number_class_false_negative  ", number_class_false_negative)

print("number_attribute_exact_match   ", number_attribute_exact_match)
print("number_attribute_false_positive    ", number_attribute_false_positive)
print("number_attribute_false_negative  ", number_attribute_false_negative)

# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN) 
# F1 = (precision * recall) / (precision + recall)

class_precision = number_class_exact_match / (number_class_exact_match + number_class_false_positive)
class_recall = number_class_exact_match / (number_class_exact_match + number_class_false_negative)
class_f1 = (class_precision * class_recall) / (class_precision + class_recall)

attribute_precision = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_positive)
attribute_recall = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_negative)
attribute_f1 = (attribute_precision * attribute_recall) / (attribute_precision + attribute_recall)

print("class_precision: ", class_precision)
print("class_recall: ", class_recall)
print("class_f1: ", class_f1)

print("attribute_precision: ", attribute_precision)
print("attribute_recall: ", attribute_recall)
print("attribute_f1: ", attribute_f1)

number_class_exact_match    28
number_class_false_positive     21
number_class_false_negative   83
number_attribute_exact_match    13
number_attribute_false_positive     68
number_attribute_false_negative   41
class_precision:  0.5714285714285714
class_recall:  0.25225225225225223
class_f1:  0.175
attribute_precision:  0.16049382716049382
attribute_recall:  0.24074074074074073
attribute_f1:  0.09629629629629628


# 2-shot use BTMS and H2S short as example

In [ ]:
two_shot_data = pd.read_csv("/content/gpt3_two_shot.csv")
two_shot_data

In [ ]:
two_shot_data_generated_list = two_shot_data["generated_text"].to_list()
two_shot_data_generated_list = two_shot_data_generated_list[1:8]

print(len(two_shot_data_generated_list))

7


In [ ]:
import parse

generated_classes = []
for index, result in enumerate(two_shot_data_generated_list):
  # Enumeration class: the substring after "Enumeration:/n" and before "\nClasses:"
  if "Enumerations:" in result:
    enumeration = result.partition("Enumerations:")[2].partition("Class")[0]
  else:
    enumeration = result.partition("Enumeration:")[2].partition("Class")[0]
  
  # Classes: the substring after "Classes:\n"
  if "Class:\n" in result:
    classes = result.partition("Class:\n")[2].partition("Relationships:")[0]
  else:
    classes = result.partition("Classes:")[2].partition("Relationships:")[0]
  
  # split string to list of string, remove empty string
  classes = (enumeration + "\n" + classes).splitlines()
  result = [i for i in classes if (i and i != "None" and i != " " and i != "")]
  generated_classes.append(result)

generated_classes

[['Repeat(weekly, monthly, every half year, yearly)',
  'TestGroup(blood test, ultrasound examination)',
  'LabTracker()',
  'Doctor(string name, int practitionerNumber, string signature)',
  'Patient(string healthNumber, string firstName, string lastName, string address, string phoneNumber, Date dateOfBirth)',
  'Test(string description, int duration, boolean appointmentRequired, boolean walkInOnly)',
  'Requisition(Date validDate, Repeat repeat)',
  'Lab(string address, int registrationNumber, double changeCancellationFee, Time startTime, Time endTime)',
  'Appointment(int confirmationNumber, Date date, Time startTime, Time endTime, Lab lab)'],
 ['EventType(birthday party, graduation party, ...)',
  'CelO()',
  'Organizer(string firstName, string lastName, string emailAddress, string postalAddress, string phoneNumber, string password)',
  'Attendee(string firstName, string lastName, string emailAddress, string password)',
  'Event(EventType eventType, Date startDate, Date endDate, st

In [ ]:
from Levenshtein import distance
from Levenshtein import ratio

number_class_exact_match = 0
number_class_false_positive = 0   # generated but is not part of the solution
number_class_false_negative = 0   # is part of the solution but is not being generated

number_attribute_exact_match = 0
number_attribute_false_positive = 0   # generated but is not part of the solution
number_attribute_false_negative = 0   # is part of the solution but is not being generated

for index in range(len(result_classes)):
  print(index)
  solution = convert_to_dict(result_classes[index])
  prediction = convert_to_dict(generated_classes[index])
  
  solution_class = list(solution.keys())
  prediction_class = list(prediction.keys())
  
  class_current_match = 0
  for c1 in solution_class:
    for c2 in prediction_class:
      # consider as exact match if the ratio > 0.9  
      if ratio(c1, c2) >= 0.9:
        # found
        print("Class: ", c1, c2)
        class_current_match += 1
        prediction_class.remove(c2)

        # find number of exact match for attribute
        attribute_current_match = 0
        attribute_solution = list(solution[c1].split(","))
        attribute_generated = list(prediction[c2].split(","))

        for a1 in attribute_solution:
          for a2 in attribute_generated:
            # consider as exact match if the ratio > 0.9  
            if a1 != "" and ratio(a1, a2) >= 0.9:
              # found
              print("Attribute: ", a1, a2)
              attribute_current_match += 1
              attribute_generated.remove(a2)
        
        number_attribute_exact_match += attribute_current_match
        number_attribute_false_positive += len(attribute_generated)
        number_attribute_false_negative += (len(attribute_solution) - attribute_current_match)

  number_class_false_positive += len(prediction_class)
  number_class_false_negative += (len(solution_class) - class_current_match)
  number_class_exact_match += class_current_match

0
Class:  LabTracker LabTracker
Class:  Patient Patient
Class:  Doctor Doctor
Attribute:  string signature  string signature
Class:  Requisition Requisition
Class:  Appointment Appointment
Attribute:   Date date  Date date
Class:  Lab Lab
Attribute:   string address string address
Class:  Test Test
1
Class:  CelO CelO
Class:  Organizer Organizer
Attribute:   string phoneNumber  string phoneNumber
Class:  Attendee Attendee
Class:  Event Event
Attribute:  string occasion  string occasion
Class:  EventType EventType
2
Class:  Position Position
Attribute:  GK GK
Attribute:   LB  LB
Class:  Player Player
Class:  Scout Scout
Class:  ScoutingAssignmnet ScoutingAssignment
Class:  PlayerProfile PlayerProfile
Attribute:  Position position Position position
3
4
Class:  OTS OTS
Class:  Subject Subject
Class:  Tutor Tutor
Attribute:  string bankAccount  string bankAccount
Class:  TutoringOffer TutoringOffer
Class:  Student Student
Class:  TutoringRequest TutoringRequest
Class:  TutoringSession Tuto

In [ ]:
print("number_class_exact_match   ", number_class_exact_match)
print("number_class_false_positive    ", number_class_false_positive)
print("number_class_false_negative  ", number_class_false_negative)

print("number_attribute_exact_match   ", number_attribute_exact_match)
print("number_attribute_false_positive    ", number_attribute_false_positive)
print("number_attribute_false_negative  ", number_attribute_false_negative)

# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN) 
# F1 = (precision * recall) / (precision + recall)

class_precision = number_class_exact_match / (number_class_exact_match + number_class_false_positive)
class_recall = number_class_exact_match / (number_class_exact_match + number_class_false_negative)
class_f1 = (class_precision * class_recall) / (class_precision + class_recall)

# recall 的分母，TP & FN = solution里attribute所有的
attribute_precision = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_positive)
attribute_recall = number_attribute_exact_match / (number_attribute_exact_match + number_attribute_false_negative)
attribute_f1 = (attribute_precision * attribute_recall) / (attribute_precision + attribute_recall)

print("class_precision: ", class_precision)
print("class_recall: ", class_recall)
print("class_f1: ", class_f1)

print("attribute_precision: ", attribute_precision)
print("attribute_recall: ", attribute_recall)
print("attribute_f1: ", attribute_f1)

number_class_exact_match    37
number_class_false_positive     24
number_class_false_negative   74
number_attribute_exact_match    13
number_attribute_false_positive     85
number_attribute_false_negative   52
class_precision:  0.6065573770491803
class_recall:  0.3333333333333333
class_f1:  0.21511627906976746
attribute_precision:  0.1326530612244898
attribute_recall:  0.2
attribute_f1:  0.07975460122699388
